In [15]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core import Environment

In [16]:
ws = Workspace.from_config()

In [17]:
experiment_name = 'livedoor-news-classification-BERT-pipeline-inference'
experiment = Experiment(ws, experiment_name)

In [18]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute Cluster の名称
amlcompute_cluster_name = "gpucluster"

# クラスターの存在確認
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    
except ComputeTargetException:
    print('指定された名称のクラスターが見つからないので新規に作成します.')
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6_V3",
                                                           max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [19]:
my_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text"])


In [20]:
project_folder = '../code/deployment'

In [21]:
envs = Environment.list(workspace=ws)
[env if env.startswith("AzureML") else None for env in envs]

['AzureML-Tutorial',
 'AzureML-Minimal',
 'AzureML-Chainer-5.1.0-GPU',
 'AzureML-PyTorch-1.2-CPU',
 'AzureML-TensorFlow-1.12-CPU',
 'AzureML-TensorFlow-1.13-CPU',
 'AzureML-PyTorch-1.1-CPU',
 'AzureML-TensorFlow-1.10-CPU',
 'AzureML-PyTorch-1.0-GPU',
 'AzureML-TensorFlow-1.12-GPU',
 'AzureML-TensorFlow-1.13-GPU',
 'AzureML-Chainer-5.1.0-CPU',
 'AzureML-PyTorch-1.0-CPU',
 'AzureML-Scikit-learn-0.20.3',
 'AzureML-PyTorch-1.2-GPU',
 'AzureML-PyTorch-1.1-GPU',
 'AzureML-TensorFlow-1.10-GPU',
 'AzureML-PyTorch-1.3-GPU',
 'AzureML-TensorFlow-2.0-CPU',
 'AzureML-PyTorch-1.3-CPU',
 'AzureML-TensorFlow-2.0-GPU',
 'AzureML-PySpark-MmlSpark-0.15',
 'AzureML-AutoML',
 'AzureML-PyTorch-1.4-GPU',
 'AzureML-PyTorch-1.4-CPU',
 'AzureML-VowpalWabbit-8.8.0',
 'AzureML-Hyperdrive-ForecastDNN',
 'AzureML-AutoML-GPU',
 'AzureML-AutoML-DNN-GPU',
 'AzureML-AutoML-DNN',
 'AzureML-Designer-R',
 'AzureML-Designer-Recommender',
 'AzureML-Designer-Transform',
 'AzureML-Designer',
 'AzureML-Designer-IO',
 'AzureML

In [22]:
env =  Environment.get(workspace=ws, name='AzureML-AutoML-DNN-GPU')

In [23]:
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
run_config.environment = env

In [24]:
from azureml.pipeline.steps import PythonScriptStep

ds_input = my_dataset.as_named_input('input1')

inferenceStep = PythonScriptStep(
    script_name="inference_script.py",
    inputs=[ds_input],
    compute_target=compute_target,
    source_directory=project_folder,
    runconfig=run_config,
    allow_reuse=True
)

In [25]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[inferenceStep])

In [26]:
pipeline.validate()
print("Pipeline validation complete")

Pipeline validation complete


In [27]:
pipeline_run = experiment.submit(pipeline)

Created step inference_script.py [cdbee0e7][32f4914a-a3bb-4c59-98eb-65598a83b76c], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun b402a232-425e-4348-83e4-2e14868860cc
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/livedoor-news-classification-BERT-pipeline-inference/runs/b402a232-425e-4348-83e4-2e14868860cc?wsid=/subscriptions/9c0f91b8-eb2f-484c-979c-15848c098a6b/resourcegroups/azureml_text/workspaces/azureml_text


In [28]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …